In [ ]:
import numpy as np
import pandas as pd
import pyesg
from series import invest_series, salary_series, age_series, mortality_sim
from asset_mix import asset_mix
config = {
        'yrs':yrs,
        'freq':freq,
        'n_scen':n_scen,
        'age':age,
        'sex':sex,
        'smoker':smoker,
        'ret_age':ret_age,
        'mi':mi,   
        'fund_value':fund_value,
        'ret_inc':ret_inc,
        'salary':salary,
        'salary_growth':salary_growth,
        'invest_pct':invest_pct,
        'inflation':inflation,
        's_pct':s_pct,
        'b_pct':b_pct,
        'pct_5':pct_5,
        's_pct_end':s_pct_end,
        'b_pct_end':b_pct_end,
        'x0':x0,
        'dt':dt,
        'n_steps':n_steps
    }

def retirement_projection(
    # projection inputs
    yrs = 40,                    # number of yrs to project
    freq = 12,                   # number of periods per year
    n_scen = 10,                 # number of scenarios

    # demographic inputs
    age = 30,                    # starting age
    sex = 'M',                   # sex
    smoker = 'NS',               # SM for smoker NS for non-smoker
    ret_age = 65,                # age at which time investments into retirement fund cease and withdrawals begin
    mi = .99,                    # mortality improvement

    # fund inputs
    fund_value = 100000,         # starting value of retirement fund
    ret_inc = 1000000,           # desired annual retirement income
    salary = 100000,             # starting salary
    salary_growth = .045,        # annual salary growth rate
    invest_pct = .15,            # pct of salary to be invested in retirement
    inflation = .02,             # annual inflation assumption
    s_pct = .80,                 # starting pct of funds to be invested in stocks  
    pct_5 = .55,                 # pct of funds to be invested in stocks 5 yrs out from retirement
    s_pct_end = .2,              # pct of funds to be invested in stocks at and during retirement
    
    # model inputs
    x0 = 100.0,                  # the start value of our process
    # random_state = 259         # optional random_state for reproducibility
    **kwargs
):
    b_pct = 1 - s_pct,           # starting pct of funds to be invested in bonds
    b_pct_end = 1 - s_pct_end    # pct of funds to be invested in bonds at and during retirement
    dt = 1/freq,                 # the length of each timestep in yrs
    n_steps = freq * yrs,        # the number of time steps per scenario
    
    x0_2 = x0
    n = n_scen

    # instantiate a new model with the required parameters
    stock_model = pyesg.GeometricBrownianMotion(mu=0.10, sigma=0.15)
    bond_model = pyesg.GeometricBrownianMotion(mu=0.05, sigma=0.05)

    # run model for both equities and bonds
    s_model_results = stock_model.scenarios(x0_2, dt,n, n_steps) 
    b_model_results = bond_model.scenarios(x0, dt, n_scen, n_steps, **kwargs)

    # create stock and bond index return arrays. 
    stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
    bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

    # set beginning of fund array to starting investment value
    stock_array = np.insert(stock_return, 0, fund_value * s_pct, axis=1)
    bond_array = np.insert(bond_return, 0, fund_value * b_pct, axis=1)

    # the last return value is not used so we add a 1 to the end to return the array to its original length
    ones_to_append = np.ones((stock_return.shape[0], 1), dtype=int)
    stock_return = np.append(stock_return, ones_to_append, axis=1)
    bond_return = np.append(bond_return, ones_to_append, axis=1)

    # create pandas series for various calcs
    salary_s = salary_series(**config)
    invest_s = invest_series(**config)
    mortality_s = mortality_sim(**config)
    asset_mix_s = asset_mix(**config)            
    age_s = age_series(**config)

    # this is where the magic happens
    # calc the fund value at each point in time, credit interest, add/withdraw from fund
    for s, b, inv, alloc_s, alloc_b in zip(stock_array, bond_array, invest_s, asset_mix_s['stock'], asset_mix_s['bond']):
        for i in range(1, len(s)):
            s[i] = s[i-1] * s[i]                            # stock fund @t = stock fund @t-1 * stock return
            b[i] = b[i-1] * b[i]                            # bond fund @t = bond fund @t-1 * bond return
            total_fund = s[i] + b[i] + invest_s[i-1]/freq   # total fund = stock + bond fund +/- investment
            s[i] = total_fund * alloc_s                     # reallocate fund value to stock fund
            b[i] = total_fund * alloc_b                     # reallocate fund value to bond fund

    total_fund = stock_array + bond_array
    df = pd.DataFrame(total_fund.T)
    return df

    # with pd.ExcelWriter('Outputs/retirement_calc.xlsx') as writer:
    #     df.to_excel(writer, sheet_name='sheet1')